In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 2.6 MB/s eta 0:00:00


In [2]:
import os
from groq import Groq
from getpass import getpass

In [3]:
os.environ['GROQ_API_KEY'] = getpass("GROQ_API_KEY")

GROQ_API_KEY··········


In [4]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))


In [5]:
# Function to call OpenAI
def get_response_v0(prompt):

    completion = client.chat.completions.create(
      model="llama-3.3-70b-versatile",
      messages=[
          {
              "role": "user",
              "content": prompt,
          },
      ],
    )

    return completion.choices[0].message.content

## Zero Shot Classification

In [6]:
complaint = (
    "The delivery guy just left the package at the gate without ringing the bell. "
    "I found it two hours later completely soaked from the rain."
)

# Without Chain of Thought Prompt
basic_prompt = f"""
Classify the following customer complaint into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Complaint: "{complaint}"

Category:"""

print(get_response_v0(basic_prompt), "\n")

Category: 1. Delivery issue. 

The customer is complaining about the way the package was delivered, specifically that it was left unattended at the gate and not handed over to them directly, which led to it getting damaged by the rain. 



## Few Shots Classification

In [7]:
# Few-shot examples
few_shot_prompt = f"""
Classify the following customer complaints into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Examples:

Complaint: "My order was supposed to arrive yesterday, but it hasn’t shown up yet."
Category: Delivery issue

Complaint: "The shoes I received had a tear on the side."
Category: Product quality

Complaint: "The support agent was rude and didn’t help at all."
Category: Customer service

Complaint: "I had trouble applying the discount code during checkout."
Category: Other

Now classify the following complaint:

Complaint: {complaint}
Category:"""

In [8]:
# Complaint to classify
complaint_to_classify = (
    "The delivery guy just left the package at the gate without ringing the bell. "
    "I found it two hours later completely soaked from the rain."
)

In [9]:
print(few_shot_prompt.format({"complaint": complaint_to_classify[0]}))


Classify the following customer complaints into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Examples:

Complaint: "My order was supposed to arrive yesterday, but it hasn’t shown up yet."
Category: Delivery issue

Complaint: "The shoes I received had a tear on the side."
Category: Product quality

Complaint: "The support agent was rude and didn’t help at all."
Category: Customer service

Complaint: "I had trouble applying the discount code during checkout."
Category: Other

Now classify the following complaint:

Complaint: The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain.
Category:


In [10]:
print(get_response_v0(few_shot_prompt.format({"complaint": complaint_to_classify[0]})))

Complaint: The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain.
Category: Delivery issue

This complaint falls under the "Delivery issue" category because it relates to the handling and delivery of the package, specifically the driver's actions and the resulting damage to the package.


## Chain of Thought

In [15]:
# Function to call OpenAI
def get_response_v2(system_prompt, prompt):

    print( "**************** Prompt *****************")
    print(system_prompt + prompt)
    print( "****************************************")

    completion = client.chat.completions.create(
      model="llama-3.3-70b-versatile",
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": prompt,
          },
      ],
    )

    return completion.choices[0].message.content

In [16]:
system_prompt = """Your job is to classify complaints into one of the following categories:

Delivery issue
Product quality
Customer service
Other

Use chain-of-thought reasoning to guide the classification of the following complaint."""

user_prompt = f"""
Complaint: {complaint}
"""

print("With Chain of Thought:")
print(get_response_v2(system_prompt,
                      user_prompt.format({"complaint":
                                         "The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain."})))

With Chain of Thought:
**************** Prompt *****************
Your job is to classify complaints into one of the following categories:

Delivery issue
Product quality
Customer service
Other

Use chain-of-thought reasoning to guide the classification of the following complaint.
Complaint: The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain.

****************************************
To classify this complaint, let's break it down step by step:

1. The complaint mentions a "delivery guy" and a "package," which suggests that the issue is related to the delivery process.
2. The specific problem mentioned is that the package was left at the gate without the delivery person notifying the customer (by ringing the bell).
3. The package was found two hours later, and it was "completely soaked from the rain," which implies that the package was damaged due to being left outside in the rain.

Considering these po

## Structured Response

In [17]:
# System prompt
system_prompt = "You are a helpful assistant that classifies customer complaints. Return the result only as a JSON object: {\"category\": \"...\"}"

In [18]:
user_prompt = f"""
Classify the customer complaint into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Complaint: {complaint}
"""

In [19]:
resp = get_response_v2(system_prompt,
                      user_prompt.format({"complaint":
                                         "The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain."}))

**************** Prompt *****************
You are a helpful assistant that classifies customer complaints. Return the result only as a JSON object: {"category": "..."}
Classify the customer complaint into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Complaint: The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain.

****************************************


In [20]:
resp

'{"category": "Delivery issue"}'

In [21]:
import json

category_dict = json.loads(resp)

print(category_dict)
print(category_dict["category"])

{'category': 'Delivery issue'}
Delivery issue


## Temperature

In [22]:
# API wrapper
def get_response_v3(system_prompt, prompt, temperature=0.7):

    print( "**************** Prompt *****************")
    print(system_prompt + prompt)
    print( "****************************************")

    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ],
        temperature=temperature
    )
    return completion.choices[0].message.content.strip()

In [23]:
system_prompt = """Your job is to classify complaints into one of the following categories:

Delivery issue
Product quality
Customer service
Other

Use chain-of-thought reasoning to guide the classification of the following complaint."""

user_prompt = f"""
Complaint: {complaint}
"""

In [24]:
print(get_response_v3(system_prompt,
                      user_prompt.format({"complaint":
                                         "The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain."})))

**************** Prompt *****************
Your job is to classify complaints into one of the following categories:

Delivery issue
Product quality
Customer service
Other

Use chain-of-thought reasoning to guide the classification of the following complaint.
Complaint: The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain.

****************************************
To classify this complaint, let's break it down step by step:

1. The complaint mentions a "delivery guy," which immediately suggests that the issue is related to the delivery process.
2. The specific problem mentioned is that the package was left at the gate without the delivery person ringing the bell, which led to the customer not being aware of the delivery.
3. As a result of the package being left unattended, it got soaked in the rain, which further emphasizes the issue with how the delivery was handled.

Considering these points, the compla

In [25]:
print(get_response_v3(system_prompt,
                      user_prompt.format({"complaint":
                                         "The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain."}),
                      temperature = 0.0))

**************** Prompt *****************
Your job is to classify complaints into one of the following categories:

Delivery issue
Product quality
Customer service
Other

Use chain-of-thought reasoning to guide the classification of the following complaint.
Complaint: The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain.

****************************************
To classify this complaint, let's break it down step by step:

1. The complaint mentions a "delivery guy," which immediately suggests that the issue is related to the delivery process.
2. The specific problem mentioned is that the package was left at the gate without the delivery person ringing the bell, which is a deviation from the expected delivery protocol.
3. The package was found two hours later, completely soaked from the rain, which implies that the package was left unattended and exposed to the elements for an extended period.
4. The com

## Self Consistency

In [26]:
import time
from collections import Counter

system_prompt = "You are a helpful assistant that classifies customer complaints. Return the result only as a JSON object: {\"category\": \"...\"}"

user_prompt = f"""
Classify the customer complaint into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Use chain-of-thought reasoning to guide the classification of the following complaint.

Complaint: {complaint}
"""

complaint = "I waited all day at home, but the package never came. When I contacted support,  they told me to wait another 48 hours without giving a reason."

print(user_prompt.format({"complaint": complaint}))

results = []

for temp in [0.0, 0.5, 2.0]:
    try:
        resp = get_response_v3(system_prompt
                              , user_prompt.format({"complaint": complaint})
                              , temperature = temp)
        category_dict = json.loads(resp)
        category = category_dict['category']
        if category is not None:
            results.append(category)
    except Exception as e:
        print(f"Error on run {temp}: {e}")

    time.sleep(1)  # to avoid rate limits

    counts = Counter(results)
    majority = counts.most_common(1)[0]

print("All predictions:", results)
print("Majority vote:", majority)


Classify the customer complaint into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Use chain-of-thought reasoning to guide the classification of the following complaint.

Complaint: The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain.

**************** Prompt *****************
You are a helpful assistant that classifies customer complaints. Return the result only as a JSON object: {"category": "..."}
Classify the customer complaint into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Use chain-of-thought reasoning to guide the classification of the following complaint.

Complaint: The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain.

****************************************
**************** Prompt *****************
You are a helpfu